In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

# Checkpoint function

In [5]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn = model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

In [6]:
def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data Load and Pre-processing function

In [9]:
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

In [24]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    # [N, 28, 28] -> [N, 28, 28, 1]
    train_data = np.expand_dims(train_data, axis = -1)
    test_data = np.expand_dims(test_data, axis = -1)
    
    train_data, test_data  = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10) # one_hot_encoding
    test_labels = to_categorical(test_labels, 10)
    
    return train_data, train_labels, test_data, test_labels

# Performance function

In [11]:
def loss_fn(model, images, labels):
    logits = model(images, training = True)
    loss =tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits = True))
    return loss

In [12]:
def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction= tf.equal(tf.argmax(logits, -1),tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

In [15]:
def grad(model, images, labels):
    with tf.GradientTape as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model, variables)

# Model function

In [29]:
def flatten():
    return tf.keras.layers.Flatten()

In [17]:
def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units= label_dim, use_bias = True, kernel_initializer = weight_init)

In [18]:
def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

# Create Model

In [21]:
class create_model_class(tf.keras.Model):
    
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten)
        
        for i in range(2):
            self.model.add(dense(256,weight_init))
            self.model.add(sigmoid())
            
        self.model.add(dense(label_dim,weight_init))
        
    def call(self, x, training =None, mask = None):
        x = self.mdoel(x)
        return x

In [22]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(sigmoid())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [25]:
# dataset
train_x, train_y, test_x , test_y = load_mnist()

In [26]:
# parameter
learning_rate = 0.001
batch_size =128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

In [27]:
# Graph Input using Dataset APi ??
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Defind model and optimizer and writer

In [30]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate =learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [ ]:
if train_flag:
    
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    cloud-load, checkpoint_counter = load(network, cehckpoint_dir)
    
    if cloud_laod:
        start_epoch = (int)(cehckpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
        
    # train phase
    with summary.writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                